In [1]:
import os
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6"
from pyspark.sql import *
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.shell import sc,spark
from pyspark.sql.functions import desc,asc,expr

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.6 (default, Sep 12 2018 18:26:19)
SparkSession available as 'spark'.


In [3]:
md = spark.read.csv("the-movies-dataset/movies_metadata.csv",header=True)
md = md.withColumn("id",expr("CAST(id AS INTEGER)"))
print(len(md.collect()))
md.printSchema

45572


<bound method DataFrame.printSchema of DataFrame[adult: string, belongs_to_collection: string, budget: string, genres: string, homepage: string, id: int, imdb_id: string, original_language: string, original_title: string, overview: string, popularity: string, poster_path: string, production_companies: string, production_countries: string, release_date: string, revenue: string, runtime: string, spoken_languages: string, status: string, tagline: string, title: string, video: string, vote_average: string, vote_count: string]>

In [24]:
from pyspark.sql.functions import expr
lm = spark.read.csv("the-movies-dataset/links_small.csv",header=True)
lm = lm.withColumn('tmdbId',expr("CAST(tmdbId AS INTEGER)"))
joinExpression = lm["tmdbId"] == md["id"]
smd = md.join(lm,joinExpression,"inner")

In [25]:
from pyspark.sql.functions import col , column,when,lit,udf,split
mergeCols = udf(lambda fruits, meat: fruits + meat)
charReplace = udf(lambda x: x.replace(u',',''))

# remove null value
smd = smd.withColumn('overview', when(smd.tagline.isNull(),'').otherwise(smd['overview']))
# smd = smd.withColumn('description',mergeCols(col('overview'),col('tagline')))
# smd = smd.withColumn('description',charReplace(col('description')))
smd = smd.withColumn("overview",split(col("overview")," ").cast("array<string>"))

In [35]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import CountVectorizer
smd.printSchema
tf = CountVectorizer(inputCol="overview",outputCol="words")
model =  tf.fit(smd.limit(10))
result = model.transform(smd.limit(10))
result.select('words').show(10,truncate=True)
# smd.printSchema
idf = IDF(inputCol="words", outputCol="features")
idfModel = idf.fit(result)
rescaledData = idfModel.transform(result)

print(rescaledData.select("features").collect())
# tokenizer = Tokenizer(inputCol="description",outputCol='words')
# wordsdata = tokenizer.transform(smd)
# hashingTF = HashingTF(inputCol='words',outputCol='rawFeatures',numFeatures=100)
# featurizedData = hashingTF.transform(wordsdata)
# featurizedData.select('rawFeatures').show(1,False)


# idf = IDF(inputCol="rawFeatures", outputCol="features")
# idfModel = idf.fit(featurizedData)
# rescaledData = idfModel.transform(featurizedData)

# tmp = rescaledData.select("label", "features").collect()
# rescaledData.select("label", "features").show()



+--------------------+
|               words|
+--------------------+
|   (319,[185],[1.0])|
|(319,[0,1,2,3,5,9...|
|(319,[0,1,2,5,6,7...|
|(319,[0,1,2,3,4,1...|
|(319,[0,1,2,3,4,5...|
|(319,[0,1,2,4,7,1...|
|(319,[1,3,8,13,17...|
|(319,[0,1,2,3,4,5...|
|(319,[0,1,2,3,4,5...|
|(319,[0,2,3,4,15,...|
+--------------------+

[Row(features=SparseVector(319, {185: 1.7047})), Row(features=SparseVector(319, {0: 0.602, 1: 0.2007, 2: 0.4013, 3: 0.6369, 5: 0.6061, 9: 2.5986, 10: 2.0232, 11: 1.2993, 12: 0.7885, 13: 2.0232, 15: 1.0116, 19: 2.5986, 23: 1.2993, 25: 1.2993, 35: 1.2993, 45: 1.7047, 51: 1.7047, 53: 1.7047, 54: 1.7047, 60: 1.7047, 63: 1.7047, 67: 1.7047, 74: 1.7047, 80: 1.7047, 83: 1.7047, 87: 1.7047, 88: 1.7047, 89: 1.7047, 90: 1.7047, 102: 1.7047, 106: 1.7047, 109: 1.7047, 117: 1.7047, 118: 1.7047, 122: 1.7047, 129: 1.7047, 134: 1.7047, 136: 1.7047, 153: 1.7047, 154: 1.7047, 186: 1.7047, 188: 1.7047, 191: 1.7047, 198: 1.7047, 199: 1.7047, 201: 1.7047, 203: 1.7047, 219: 1.7047, 222: 1.7

In [16]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Spark"),
    (0.0, "I wish Java could use case classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
wordsData = tokenizer.transform(sentenceData)
wordsData.show(10,False)
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=100)

featurizedData = hashingTF.transform(wordsData)
featurizedData.select('rawFeatures').show(10,False)
# # alternatively, CountVetorizer can also be used to get term frequency vectors

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

tmp = rescaledData.select("label", "features").collect()
rescaledData.select("label", "features").show(10,False)
# print(tmp)



+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+

+---------------------------------------------------------+
|rawFeatures                                              |
+---------------------------------------------------------+
|(100,[5,29,57,60,77],[1.0,1.0,1.0,1.0,1.0])              |
|(100,[9,13,29,42,67,89,95],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|(100,[4,38,86,93,95],[1.0,1.0,1.0,1.0,1.0])              |
+-----------------------------

In [ ]:
from pyspark.mllib.feature import HashingTF, IDF

# Load documents (one per line).
documents = sc.textFile("data/mllib/kmeans_data.txt").map(lambda line: line.split(" "))

hashingTF = HashingTF()
tf = hashingTF.transform(documents)

# While applying HashingTF only needs a single pass to the data, applying IDF needs two passes:
# First to compute the IDF vector and second to scale the term frequencies by IDF.
tf.cache()
idf = IDF().fit(tf)
tfidf = idf.transform(tf)

# spark.mllib's IDF implementation provides an option for ignoring terms
# which occur in less than a minimum number of documents.
# In such cases, the IDF for these terms is set to 0.
# This feature can be used by passing the minDocFreq value to the IDF constructor.
idfIgnore = IDF(minDocFreq=2).fit(tf)
tfidfIgnore = idfIgnore.transform(tf)

In [9]:
from pyspark.ml.feature import Word2Vec

# Input data: Each row is a bag of words from a sentence or document.
documentDF = spark.createDataFrame([
    ("Hi I heard about Spark".split(" "), ),
    ("I wish Java could use case classes".split(" "), ),
    ("Logistic regression models are neat".split(" "), )
], ["text"])
documentDF.show(10,False)
# Learn a mapping from words to Vectors.
word2Vec = Word2Vec(vectorSize=3, minCount=0, inputCol="text", outputCol="result")
model = word2Vec.fit(documentDF)

result = model.transform(documentDF)
for row in result.collect():
    text, vector = row
    print("Text: [%s] => \nVector: %s\n" % (", ".join(text), str(vector)))

+------------------------------------------+
|text                                      |
+------------------------------------------+
|[Hi, I, heard, about, Spark]              |
|[I, wish, Java, could, use, case, classes]|
|[Logistic, regression, models, are, neat] |
+------------------------------------------+

Text: [Hi, I, heard, about, Spark] => 
Vector: [-0.0190316129475832,-0.0027922827750444415,0.008608809486031533]

Text: [I, wish, Java, could, use, case, classes] => 
Vector: [0.05155504494905472,-0.028778317916606153,0.021439425381166593]

Text: [Logistic, regression, models, are, neat] => 
Vector: [-0.002297615259885788,0.05957087948918343,0.05615671277046204]



In [15]:
from pyspark.ml.feature import CountVectorizer

# Input data: Each row is a bag of words with a ID.
df = spark.createDataFrame([
    (0, "a b c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

df.select("words").show()
# fit a CountVectorizerModel from the corpus.
cv = CountVectorizer(inputCol="words", outputCol="features")

model = cv.fit(df)

result = model.transform(df)
result.show(truncate=False)
df.printSchema

+---------+
|    words|
+---------+
|    a b c|
|a b b c a|
+---------+



IllegalArgumentException: 'requirement failed: Column words must be of type equal to one of the following types: [ArrayType(StringType,true), ArrayType(StringType,false)] but was actually of type StringType.'